In [5]:
%pip install requests
%pip install flask
%pip install python-dotenv

import os
import requests
import glob
from pprint import pprint
from dotenv import load_dotenv
load_dotenv()


#Retrieve the URL of the API and the API key
API_URL = os.getenv("API_URL")
assert API_URL

API_KEY = os.getenv("API_KEY")
assert API_KEY

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Construct the URL of the `/indexes` endpoint
INDEXES_URL = f"{API_URL}/indexes"

# Set the header of the request
default_header = {
    "x-api-key": API_KEY
}

# Define a function to create an index with a given name
def create_index(index_name, index_options, engine):
    # Declare a dictionary named data
    data = {
        "engine_id": engine,
        "index_options": index_options,
        "index_name": index_name,
    }

    # Create an index
    response = requests.post(INDEXES_URL, headers=default_header, json=data)

    # Store the unique identifier of your index
    INDEX_ID = response.json().get('_id')

    # Check if the status code is 201 and print success
    if response.status_code == 201:
        print(f"Status code: {response.status_code} - The request was successful and a new index was created.")
    else:
        print(f"Status code: {response.status_code}")
    pprint(response.json())
    return INDEX_ID


# Create the indexes
index_id_content_classification = create_index(index_name = "insta+tiktok", index_options=["visual", "conversation", "text_in_video", "logo"], engine = "marengo2.5")

# Print the created index IDs
print(f"Created index IDs: {index_id_content_classification}")

Status code: 201 - The request was successful and a new index was created.
{'_id': '6760721bf3867b4dd07ceec0'}
Created index IDs: 6760721bf3867b4dd07ceec0


In [ ]:
# with local folder
import os
import requests
from concurrent.futures import ThreadPoolExecutor

TASKS_URL = f"{API_URL}/tasks"
TASK_ID_LIST = []
video_folder = 'classify'  # folder containing the video files
INDEX_ID = '6760721bf3867b4dd07ceec0'

def upload_video(file_name):
    # Validate if a video already exists in the index
    task_list_response = requests.get(
        TASKS_URL,
        headers=default_header,
        params={"index_id": INDEX_ID, "filename": file_name},
    )
    if "data" in task_list_response.json():
        task_list = task_list_response.json()["data"]
        if len(task_list) > 0:
            if task_list[0]['status'] == 'ready': 
                print(f"Video '{file_name}' already exists in index {INDEX_ID}")
            else:
                print("task pending or validating")
            return

    # Proceed further to create a new task to index the current video if the video didn't exist in the index already
    print("Entering task creation code for the file: ", file_name)
    
    if file_name.endswith('.mp4'):  # Make sure the file is an MP4 video
        file_path = os.path.join(video_folder, file_name)  # Get the full path of the video file
        with open(file_path, "rb") as file_stream:
            data = {
                "index_id": INDEX_ID,
                "language": "en"
            }
            file_param = [
                ("video_file", (file_name, file_stream, "application/octet-stream")),] #The video will be indexed on the platform using the same name as the video file itself.
            response = requests.post(TASKS_URL, headers=default_header, data=data, files=file_param)
            TASK_ID = response.json().get("_id")
            TASK_ID_LIST.append(TASK_ID)
            # Check if the status code is 201 and print success
            if response.status_code == 201:
                print(f"Status code: {response.status_code} - The request was successful and a new resource was created.")
            else:
                print(f"Status code: {response.status_code}")
            print(f"File name: {file_name}")
            pprint(response.json())
            print("\n")

# Get list of video files
video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]

# Create a ThreadPoolExecutor
with ThreadPoolExecutor() as executor:
    # Use executor to run upload_video in parallel for all video files
    executor.map(upload_video, video_files)

Entering task creation code for the file:  ssstik.io_@happy.huhn_1734417697208.mp4
Status code: 201 - The request was successful and a new resource was created.
File name: ssstik.io_@happy.huhn_1734417697208.mp4
{'_id': '67611e31f3867b4dd07ceec9'}




In [ ]:
# with download link
import os
import requests
from concurrent.futures import ThreadPoolExecutor
from io import BytesIO
from urllib.parse import urlparse

TASKS_URL = f"{API_URL}/tasks"
TASK_ID_LIST = []
INDEX_ID = '6760721bf3867b4dd07ceec0'

def upload_video_from_url(video_url):
    # Extract filename from URL
    filename = os.path.basename(urlparse(video_url).path)
    if not filename.endswith('.mp4'):
        filename += '.mp4'  # Add .mp4 extension if not present
    
    # Validate if video already exists in the index
    task_list_response = requests.get(
        TASKS_URL,
        headers=default_header,
        params={"index_id": INDEX_ID, "filename": filename},
    )
    
    if "data" in task_list_response.json():
        task_list = task_list_response.json()["data"]
        if len(task_list) > 0:
            if task_list[0]['status'] == 'ready':
                print(f"Video '{filename}' already exists in index {INDEX_ID}")
            else:
                print("Task pending or validating")
            return

    print("Entering task creation code for the URL: ", video_url)
    
    try:
        # Download the video content
        video_response = requests.get(video_url, stream=True)
        video_response.raise_for_status()  # Raise an exception for bad status codes
        
        # Create a BytesIO object from the downloaded content
        video_stream = BytesIO(video_response.content)
        
        # Prepare the upload
        data = {
            "index_id": INDEX_ID,
            "language": "en"
        }
        
        file_param = [
            ("video_file", (filename, video_stream, "application/octet-stream")),
        ]
        
        # Upload the video
        response = requests.post(TASKS_URL, headers=default_header, data=data, files=file_param)
        TASK_ID = response.json().get("_id")
        TASK_ID_LIST.append(TASK_ID)
        
        # Check response
        if response.status_code == 201:
            print(f"Status code: {response.status_code} - The request was successful and a new resource was created.")
        else:
            print(f"Status code: {response.status_code}")
        print(f"File name: {filename}")
        pprint(response.json())
        print("\n")
        
    except requests.exceptions.RequestException as e:
        print(f"Error downloading or uploading video from {video_url}: {str(e)}")

# Example usage with list of URLs
video_urls = [
    'https://tikcdn.io/ssstik/7402995674268159250'
]

# Create a ThreadPoolExecutor
with ThreadPoolExecutor() as executor:
    # Use executor to run upload_video_from_url in parallel for all video URLs
    executor.map(upload_video_from_url, video_urls)

Entering task creation code for the URL:  https://tikcdn.io/ssstik/7402995674268159250
Status code: 201 - The request was successful and a new resource was created.
File name: 7402995674268159250.mp4
{'_id': '676121cbf3867b4dd07ceecc'}




In [11]:
import time
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed

def monitor_upload_status(task_id):
    TASK_STATUS_URL = f"{API_URL}/tasks/{task_id}"
    while True:
        response = requests.get(TASK_STATUS_URL, headers=default_header)
        STATUS = response.json().get("status")
        if STATUS == "ready":
            return task_id, STATUS
        remain_seconds = response.json().get('process', {}).get('remain_seconds', None)
        upload_percentage = response.json().get('process', {}).get('upload_percentage', None)
        if remain_seconds is not None:
            print(f"Task ID: {task_id}, Remaining seconds: {remain_seconds}, Upload Percentage: {upload_percentage}")
        else:
             print(f"Task ID: {task_id}, Status: {STATUS}")
        time.sleep(10)

# Define starting time
start = time.time()
print("Starting to monitor...")

# Monitor the indexing process for all tasks
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(monitor_upload_status, task_id) for task_id in TASK_ID_LIST}
    with open('upload_status.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Task ID", "Status"])
        for future in as_completed(futures):
            task_id, status = future.result()
            writer.writerow([task_id, status])

# Define ending time
end = time.time()
print("Monitoring finished")
print("Time elapsed (in seconds): ", end - start)

Starting to monitor...
Task ID: 67611e31f3867b4dd07ceec9, Status: pending
Task ID: 67611e31f3867b4dd07ceec9, Status: pending
Task ID: 67611e31f3867b4dd07ceec9, Status: pending
Task ID: 67611e31f3867b4dd07ceec9, Status: pending
Task ID: 67611e31f3867b4dd07ceec9, Status: pending
Task ID: 67611e31f3867b4dd07ceec9, Status: pending
Task ID: 67611e31f3867b4dd07ceec9, Status: pending
Task ID: 67611e31f3867b4dd07ceec9, Status: pending
Task ID: 67611e31f3867b4dd07ceec9, Status: indexing
Task ID: 67611e31f3867b4dd07ceec9, Status: indexing
Task ID: 67611e31f3867b4dd07ceec9, Status: indexing
Task ID: 67611e31f3867b4dd07ceec9, Status: indexing
Task ID: 67611e31f3867b4dd07ceec9, Status: indexing
Task ID: 67611e31f3867b4dd07ceec9, Status: indexing
Task ID: 67611e31f3867b4dd07ceec9, Status: indexing
Task ID: 67611e31f3867b4dd07ceec9, Status: indexing
Task ID: 67611e31f3867b4dd07ceec9, Status: indexing
Task ID: 67611e31f3867b4dd07ceec9, Status: indexing
Monitoring finished
Time elapsed (in seconds):  1

In [13]:
# List all the videos in an index
INDEX_ID='6760721bf3867b4dd07ceec0'
default_header = {
    "x-api-key": API_KEY
}
# INDEX_ID='64502d238b1dd6cde172a9c5' #movies
# INDEX_ID= '64399bc25b65d57eaecafb35' #lex
INDEXES_VIDEOS_URL = f"{API_URL}/indexes/{INDEX_ID}/videos?page_limit=20"
response = requests.get(INDEXES_VIDEOS_URL, headers=default_header)

response_json = response.json()
pprint(response_json)

video_id_name_list = [{'video_id': video['_id'], 'video_name': video['metadata']['filename']} for video in response_json['data']]

pprint(video_id_name_list)

{'data': [{'_id': '676121cb183e93cba5f0fc66',
           'created_at': '2024-12-17T07:01:31Z',
           'metadata': {'audio_channel': 2,
                        'audio_length_ns': 9200000000,
                        'audio_sample_rate': 44100,
                        'audio_stream_id': '0x2',
                        'audio_stream_idx': 1,
                        'duration': 9.1000001,
                        'engine_ids': ['marengo2.5'],
                        'filename': '7402995674268159250.mp4',
                        'fps': 30,
                        'height': 1280,
                        'size': 1266809,
                        'video_length_ns': 9100000100,
                        'video_stream_id': '0x1',
                        'video_stream_idx': 0,
                        'video_title': '7402995674268159250.mp4',
                        'width': 720}},
          {'_id': '67611e318eb48e4f74e62173',
           'created_at': '2024-12-17T06:46:09Z',
           'indexed_at':

In [26]:
# Construct the URL of the `/classify/bulk` endpoint
CLASSIFY_BULK_URL = f"{API_URL}/classify/bulk"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

# Declare a dictionary named `data`
data =  {
    "options": ["conversation", "text_in_video"],
    "index_id": INDEX_ID,
    "video_ids": [
        "676121cb183e93cba5f0fc66",
        "67611e318eb48e4f74e62173"
    ], 
    "classes": [
        {
            "name": "BeautyTok",
            "prompts": [
                "Makeup",
                "Skincare",
                "cosmetic products",
                "doing nails",
                "doing hair",
                "DanceTok",
                "dance tutorial",
                "dance competition",
                "dance challenge",
                "dance trend",
                "dancing with friends"
            ]
        },
        {
            "name": "CookTok",
            "prompts": [
                "cooking tutorial",
                "cooking utensils",
                "baking tutorials",
                "recipes",
                "restaurants",
                "food",
                "pasta"
            ]
        },
        {
            "name": "AnimalTok",
            "prompts": [
                "dog",
                "cat",
                "birds",
                "fish",
                "playing with pets",
                "pets doing funny things"
            ]
        },
        {
            "name": "ArtTok",
            "prompts": [
                "handicraft",
                "drawing",
                "graffiti",
                "sketching",
                "digital art",
                "coloring",
                "sketchbook",
                "artwork",
                "artists"
            ]
        }
    ]
}

# Make a classification request
response = requests.post(CLASSIFY_BULK_URL, headers=headers, json=data)
print(f'Status code: {response.status_code}')
pprint(response.json())

Status code: 400
{'code': 'classify_prompts_limit_exceeded',
 'message': 'The number of prompts exceeds the limit of 0.'}
